In [2]:
!pip install spacy
!pip install pandas
!pip install spacytextblob
!pip install bs4
!pip install gensim==3.8.3


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'


  DEPRECATION: bs4 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  DEPRECATION: gensim is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  Running setup.py install for gensim did not run successfully.
  exit code: 1
  
  [638 lines of output]
  running install
  C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    warnings.warn(
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\gensim
  copying gensim\downloader.py -> build\lib.win-amd64-cpython-311\gensim
  copying gensim\interfaces.py


     ---------------------------------------- 23.4/23.4 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1
  Running setup.py install for gensim: started
  Running setup.py install for gensim: finished with status 'error'
  Rolling back uninstall of gensim
  Moving to c:\users\gabri\appdata\local\programs\python\python311\lib\site-packages\gensim-4.3.1.dist-info\
   from C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\~ensim-4.3.1.dist-info
  Moving to c:\users\gabri\appdata\local\programs\python\python311\lib\site-packages\gensim\
   from C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\~ensim


In [55]:
!pip install matplotlib
!pip install scipy
!pip install newspaper3k


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download en_core_web_sm


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 12.8/12.8 MB 6.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [56]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
from bs4 import BeautifulSoup
import requests
from newspaper import Article

In [24]:
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import FastText
#from gensim.models.wrappers import LdaMallet

import numpy as np
import os
from random import shuffle
import re
#import fasttext
#import fasttext.util
#from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt 
import numpy as np
#import pyLDAvis
#import pyLDAvis.gensim

In [59]:
# Loading our two pipelines
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

config = {"punct_chars": [".", "?", "!", "。"]}
nlp.add_pipe('sentencizer')

In [67]:
# Using panda to load in our .csv file
df = pd.read_csv("urls.csv") # File we are checking
urls = df["Address"].tolist() # Column name we are checking
url_sent_score = []
url_sent_label = []
url_subj_score = []
url_subj_label = []
total_pos = []
total_neg = []
topic_sentiment = []

In [68]:
def get_named_entities(doc):
    ent_list = []
    for ent in doc.ents:
        ent_list.append((ent.text,ent.label_))
    return ent_list

def get_sentences(doc):
    return doc.sents

def get_sentence_list(doc):
    return [sent for sent in doc.sents]

#Takes a doc object from spacy and returns a tuple list of form (sentence, sentiment of sentence) for all sentences
def sentence_sentiment_from_doc(doc):
    sentences = get_sentences(doc)
    tuple_list = []
    for sentence in sentences:
        sent_doc = nlp(sentence.text)
        tuple_list.append((sentence.text,sent_doc._.blob.polarity))
    return tuple_list

#Finds all named entities (does not filter by person, date, etc) and returns sentiment value based on the sentences they
#are included in. Returns dictionary of form {entity name : sentiment value}
def sentence_sentiment_on_named_entities(doc):
    named_ent_list = get_named_entities(doc)
    sentence_sentiment_list = sentence_sentiment_from_doc(doc)
    return_dict = {}
    
    for (name, label) in named_ent_list:
        #print(name, "\n")
        #print(type(name))
        return_dict[name] = 0
        
        for (sentence, sentiment) in sentence_sentiment_list:
             if sentence.find(name) != -1:
                    #print(name,sentence)
                    return_dict[name] += sentiment
    
    return return_dict

#takes in a single topic word, the word's weight, and the doc, and returns sentiment of that word within the doc
def sentence_level_sentiment_of_word(word, weight, doc):
    sentence_sentiment_list = sentence_sentiment_from_doc(doc)
    sentiment_total = 0
    
    for (sentence, sentiment) in sentence_sentiment_list:
             if sentence.find(word) != -1:
                    #print(name,sentence)
                    sentiment_total += sentiment
    return sentiment_total

def weighted_word_sentiment(word,weight, doc):
    sentiment = sentence_level_sentiment_of_word(word, weight, doc)
    weighted_sentiment = sentiment * weight
    return weighted_sentiment

In [69]:
#https://pypi.org/project/spacytextblob/
for count, x in enumerate(urls):
    if(count % 10 == 0):
        print(count)
    try:
        url = x
        
        article = Article(url)
        article.download()
        article.parse()
#         headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
#         res = requests.get(url,headers=headers)
#         html_page = res.text

#         soup = BeautifulSoup(html_page, 'html.parser')
#         for script in soup(["script", "style","meta","label","header","footer"]):
#             script.decompose()
#         page_text = (soup.get_text()).lower()
#         page_text = page_text.strip().replace("  ","")
#         page_text = "".join([s for s in page_text.splitlines(True) if s.strip("\r\n")])
        
        page_text = (article.text).lower()
        page_text = page_text.strip().replace("  ","")
        page_text = "".join([s for s in page_text.splitlines(True) if s.strip("\r\n")])
        
        # Start the sentiment analysis now
        doc = nlp(page_text)
        sentiment = doc._.blob.polarity
        sentiment = round(sentiment,2)
        subjectivity = doc._.blob.subjectivity
        subjectivity = round(subjectivity,2)
        
#         if count / 3 == 1:
#             print("enter test spot\n")
#             #ent_list = get_named_entities(doc)
#             #print(ent_list)
#             sentence_sentiment_list = sentence_sentiment_from_doc(doc)
#             #sent_1 = sentence_sentiment_list.pop()
#             #print(sent_1)
#             #print(type(sent_1[0]))
#             print(sentence_sentiment_list)
#             topic_level_sentiment = sentence_sentiment_on_named_entities(doc)
#             #print(topic_level_sentiment)
#             print(sentence_level_sentiment_of_word('kansas',0,doc))
#             break
            
        #print("\n")

        # Gives positive or negative label
        if sentiment >= -0.033 and sentiment <= 0.043:
            sent_label = "Neutral"
        elif sentiment > 0.043 and sentiment < 0.143:
            sent_label = "Neutral Positive"
        elif sentiment > 0.143:
            sent_label = "Positive"
        elif sentiment < -0.033 and sentiment > -0.062:
            sent_label = "Neutral Negative"
        elif sentiment < -0.062:
            sent_label = "Negative"

        #Document's topic-level sentiment assessment of named entities
        topic_level_sentiment = sentence_sentiment_on_named_entities(doc)
        topic_sentiment.append(topic_level_sentiment)
        
        #Adding label, sentiment, subjectivity of document
        url_sent_label.append(sent_label)
        url_sent_score.append(sentiment)
        url_subj_score.append(subjectivity)

        positive_words = []
        negative_words = []

        for x in doc._.blob.sentiment_assessments.assessments:
          if x[1] > 0:
            positive_words.append(x[0][0])
          elif x[1] < 0:
            negative_words.append(x[0][0])
          else:
            pass

        total_pos.append(', '.join(set(positive_words)))
        total_neg.append(', '.join(set(negative_words)))
    except:
        url_sent_label.append("Error")
        url_sent_score.append(0.0)
        url_subj_score.append(0.0)
        topic_sentiment.append({'Error':0})

        positive_words = []
        negative_words = []

        total_pos.append(', '.join(set(positive_words)))
        total_neg.append(', '.join(set(negative_words)))

print("The average sentiment score was: " + str(sum(url_sent_score) / len(url_sent_score)))
print("The average subjectivity score was: " + str(sum(url_subj_score) / len(url_subj_score)))

df["Sentiment Score"] = url_sent_score
df["Sentiment Label"] = url_sent_label
df["Subjectivity Score"] = url_subj_score
df["Positive Words"] = total_pos
df["Negative Words"] = total_neg
df["Topic Sentiment"] = topic_sentiment

df

0
10
20
30
The average sentiment score was: 0.022580645161290328
The average subjectivity score was: 0.2225806451612903


,Address,Sentiment Score,Sentiment Label,Subjectivity Score,Positive Words,Negative Words,Topic Sentiment
0,https://www.thegatewaypundit.com/2022/05/world...,0.05,Neutral Positive,0.28,"absolute, major, not, thanks, new, more, free,...","other, absolutely, needless, past, single","{'this week': 0.1325, 'davos': 0.1325, 'switze..."
1,https://gettr.com/post/ptt4ta7c84,0.00,Neutral,0.00,,,{}
2,https://komonews.com/news/coronavirus/if-covid...,0.00,Neutral,0.00,,,{}
3,https://www.lewrockwell.com/2018/11/no_author/...,0.10,Neutral Positive,0.41,"better, new, modern, clean, aptly, major, firs...","other, waste, base, military, simplistic, late...","{'spanish': 0.6340909090909091, 'an estimated ..."
4,https://nation.com.pk/08-Apr-2021/russia-offer...,0.07,Neutral Positive,0.28,"economic, good, particularly, new, more, direc...","mainly, actively, foreign, military","{'islamabad': 0.2583333333333333, 'russia': 1...."
5,https://barrie.ctvnews.ca/cfb-borden-based-mil...,-0.02,Neutral,0.38,"first, good, unique, not, willingly, fine, full","other, due, wrong, very, not, base, sorry, mil...","{'canadian': -0.5625, 'covid-19': -0.0625, 'ke..."
6,https://www.reuters.com/world/us/us-army-disch...,0.07,Neutral Positive,0.42,"more, win, first, latest","other, least, roughly, active, military","{'washington': 0.0, 'feb 2': 0.0, 'reuters': 0..."
7,https://www.dailysabah.com/world/europe/sweden...,0.03,Neutral,0.30,"first, light, directly, new, direct, kind","active, green, previously, long, military, for...","{'sweden': 0.23333333333333334, 'sunday': -0.0..."
8,https://www.military.com/daily-news/2022/11/01...,0.07,Neutral Positive,0.46,"sincere, developed, legal, most, best, new, mo...","behind, active, previous, limited, base, milit...","{'covid-19': 1.1511111111111112, 'this summer'..."
9,https://roundingtheearth.substack.com/p/defini...,0.09,Neutral Positive,0.41,"advanced, better, particularly, profound, extr...","terrifying, other, least, not, serious, previo...","{'congress': 0.0625, 'american': 0.35416666666..."
